In [29]:
# import all libraries
import pandas as pd
import numpy as np
from lets_plot import *
LetsPlot.setup_html()

# Data Gathering / Cleaning

We can gather all data from our simulations results, clean up the data for easier access, and split properties to make access easier

In [30]:
# loading SIM results
df = pd.read_csv('battle_results.csv')

# split results to bools
df['p1win'] = df['result'] == 1
df['p2win'] = df['result'] == 2
df['tie'] = df['result'] == 0

# standardize names in Title Case
df['p1name'] = df['p1name'].str.title()
df['p2name'] = df['p2name'].str.title()

# drop CSV index column
df = df.drop(columns=['Unnamed: 0'])

# view first few rows to understand data shape
df.head(5)

,player1,player2,result,p1name,p2name,p1win,p2win,tie
0,0,0,2,Scyther,Scyther,False,True,False
1,0,1,2,Scyther,Pinsir,False,True,False
2,0,2,2,Scyther,Moltres,False,True,False
3,0,3,2,Scyther,Moltres Ex,False,True,False
4,0,4,2,Scyther,Heatmor,False,True,False


We now want to gather the individual pokemon data from base CSV database used for the simulations, so that we can merge relevant data from it into out battle results for better analysis and charts

In [31]:
pokemon_df = pd.read_csv('../simulation/data/pokemon-tcg-with-fossils.csv')
pokemon_df.head(5)

,id,stage,name,from,hp,type,ability,a1cost,a1name,a1damage,a1bonus,a1bonusDmg,a2cost,a2name,a2damage,a2bonus,a2bonusDmg,weakness,retreat,rarity
0,ga1,0,bulbasaur,NaN,70,grass,NaN,gx,vine whip,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fire,1,1d
1,ga2,1,ivysaur,bulbasaur,90,grass,NaN,gxx,razor leaf,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fire,2,2d
2,ga3,2,venusaur,ivysaur,160,grass,NaN,ggxx,mega drain,80.0,heal,30.0,NaN,NaN,NaN,NaN,NaN,fire,3,3d
3,ga4,2,venusaur EX,ivysaur,190,grass,NaN,gxx,razor leaf,60.0,NaN,NaN,ggxx,giant bloom,100.0,heal,30.0,fire,3,4d
4,ga5,0,caterpie,NaN,50,grass,NaN,x,find a friend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fire,1,1d


And then here we can collapse our battle results into a lookup table of evolution-line IDs and the maximum evolution's name

In [32]:
evo_names = df.groupby('player1')['p1name'].first().reset_index().rename(columns={'player1': 'id', 'p1name': 'name'})
evo_names.head(5)

,id,name
0,0,Scyther
1,1,Pinsir
2,2,Moltres
3,3,Moltres Ex
4,4,Heatmor


We also want to calculate the total number of battles each Pokemon has. This is given as

$$B=\text{count}(p_{id}) \times 2 -1$$

where $B$ is the number of total battles per pokemon, and $\text{count}(p_{id})$ is a count of the total number of Pokemon in the set. Every pokemon battles every other pokemon, so we double the number of battles they experience as the Player 1, then subtract 1 for the battle they face against themself.

In [33]:
max_battles = int((len(df['player1'].unique())) * 2) - 1
max_battles

615

We can now combine all the results to aggregate the SIM results into individual performance metrics:

In [34]:
# Pokemon wins as player-1
player_wins = df.groupby('player1')[['p1win', 'tie']].sum().rename(columns={'p1win': 'player_win', 'tie': "player_tie"})

# Pokemon wins as player-2
opponent_wins = df.groupby('player2')[['p2win', 'tie']].sum().rename(columns={'p2win': 'opponent_win', 'tie': "opponent_tie"})

# Total player-1 and -2 metrics per pokemon
wins = player_wins.merge(opponent_wins, left_index=True, right_index=True).reset_index().rename(columns={'player1': 'id'})

# Aggregate total wins, ties, and losses
wins['wins'] = wins['player_win'] + wins['opponent_win']
wins['ties'] = wins['player_tie'] + wins['opponent_tie']
wins['losses'] = max_battles - (wins['wins'] + wins['ties'])

# Merge with pokemon Evolution names
wins = wins.set_index('id').merge(evo_names.set_index('id'), right_index=True, left_index=True).reset_index()

# Win percentage metric
wins['win_perc'] = (wins['wins'] / max_battles * 100).round(2)

# Boolean if it is an EX Pokemon
wins['is_ex'] = wins['name'].str.contains(' Ex')

# Get the Index of the Pokemon in the individual data frame
wins['pk_index'] = wins['name'].apply(lambda name: pokemon_df[pokemon_df['name'].str.lower() == name.lower()].first_valid_index())

# Join on the pk_index to pull features for analysis
wins['type'] = wins['pk_index'].apply(lambda id: pokemon_df.loc[id, 'type'].title())
wins['stage'] = wins['pk_index'].apply(lambda id: {0: 'Basic', 1: 'Stage 1', 2: 'Stage 2'}[pokemon_df.loc[id, 'stage']])
wins['ability'] = wins['pk_index'].apply(lambda id: pokemon_df.loc[id, 'ability'].title() if pokemon_df.loc[id, 'ability'] is not np.nan else 'None')
wins['has_ability'] = wins['pk_index'].apply(lambda id: pokemon_df.loc[id, 'ability'] is not np.nan)
wins['hp'] = wins['pk_index'].apply(lambda id: pokemon_df.loc[id, 'hp'])

# View first 5 rows of data
wins.head(5)

,id,player_win,player_tie,opponent_win,opponent_tie,wins,ties,losses,name,win_perc,is_ex,pk_index,type,stage,ability,has_ability,hp
0,0,45,0,88,0,133,0,482,Scyther,21.63,False,24,Grass,Basic,None,False,70
1,1,140,0,196,0,336,0,279,Pinsir,54.63,False,25,Grass,Basic,None,False,90
2,2,146,0,231,0,377,0,238,Moltres,61.30,False,45,Fire,Basic,None,False,100
3,3,204,0,258,0,462,0,153,Moltres Ex,75.12,True,46,Fire,Basic,None,False,140
4,4,63,0,105,0,168,0,447,Heatmor,27.32,False,47,Fire,Basic,None,False,80


In [35]:
# function to get all lost battles from a Pokemon
def get_losses(id:int) -> set[int]:
    p1 = df[(df['player1'] == id) & ~(df['p1win'])]
    p2 = df[(df['player2'] == id) & ~(df['p2win'])]
    return set([int(item) for item in set(p1['player2'].unique()) | set(p2['player1'].unique())])

Both unique Exeggutor EX evolution lines tend to be top performers in the battles, likely due to the fact that Exeggutor EX has the following traits:

|Trait|Benefit|
|:----|:------|
|180hp|Higher HP than ~90% of all Pokemon, survives longer|
|Attack cost 1 grass energy|Can start attacking immediately without needing to wait several turns to ramp up damage and energy|
|Attack deals 40-80 damage|Deals a medium amount of damage based on a coin flip, and can occasionally get very lucky coin flips, and deal very high damage every single turn|

The following represent the two unique evolution lines:

|Evolution ID|Basic (ID)|Stage 1 (ID)|Difference|
|:-----------|:--------:|:----------:|:---------|
|74|Exeggcute (ga21)|Exeggutor EX (ga23)| Exeggcute attack deals 20 damage |
|75|Exeggcute (mi1)|Exeggutor EX (ga23)| Exeggcute attack deals no damage, and adds 1 grass energy |

Below we grab the difference in loss statistics between the two lines:


In [36]:
loss74 = get_losses(74)
loss75 = get_losses(75)

# Union of loss sets
eggLosses = loss74 & loss75

named_evo_list = lambda s: [evo_names.loc[item, 'name'].title() for item in s]

print(f"Common losses:           {named_evo_list(eggLosses)}")
print(f"Egg 74 unique losses:    {named_evo_list(loss74 - eggLosses)}")
print(f"Egg 75 unique losses:    {named_evo_list(loss75 - eggLosses)}")

Common losses:           ['Exeggutor Ex', 'Rampardos', 'Bastidon', 'Aerodactyl Ex', 'Rapidash', 'Infernape Ex']
Egg 74 unique losses:    ['Magnezone', 'Bellossom', 'Pidgeot', 'Rapidash']
Egg 75 unique losses:    ['Glaceon', 'Exeggutor Ex', 'Mismagius Ex', 'Pidgeot Ex', 'Bellossom', 'Gengar Ex', 'Salazzle']


While Exeggutor EX will not always have the exact same losses through the simulation, here are several common losses, and possible reasons why this is a common loss:

### Common

|Line|Reason|
|:---|:-----|
|Rampardos|A high-HP pokemon with a 1-energy attack that deals 130 damage, so it will always 2-shot Exeggutor EX|
|Bastidon|A high HP pokemon that has an ability that causes all attacks to only have a 50% chance of dealing any damage|
|Aerodactyl EX|Has an ability that prevents the active Pokemon from evolving, so when it is the first player, it essentially only has to beat 1 Exeggutor EX|
|Rapidash|Has type-advantage on moves against Exeggutor, and deals a total of 60-120 damage every turn|
|Infernape EX|Once its 2-energy attach ramps up, it can KO an exeggutor in a single hit|
|Mismagius EX|Adds a status on Exeggutor that makes all attacks only have a 50% chance of hitting|

### Evo-Line 74

Tends to not have many other losses, likely due to the fact that its pre-evolution deals 20 damage with its first attack, which is a breaking point to KO several pokemon 1-turn earlier.

### Evo-Line 75

|Line|Reason|
|:---|:-----|
|Exeggutor|The exeggute pre-evolution adds one additional energy on the Exeggutor EX, and Exeggutor deals damage based on the total energy attached to the Pokemon|

# Results

Below, we have compiled the top 20 pokemon evolution lines based on their win-rate percentage

In [37]:
top_20 = wins[['id', 'wins', 'win_perc', 'name']].sort_values('wins', axis=0, ascending=False).head(20)
top_20

,id,wins,win_perc,name
74,74,605,98.37,Exeggutor Ex
75,75,602,97.89,Exeggutor Ex
300,300,597,97.07,Rampardos
289,289,591,96.10,Magnezone
290,290,584,94.96,Magnezone
286,286,582,94.63,Infernape Ex
176,176,582,94.63,Aerodactyl Ex
99,99,569,92.52,Starmie Ex
72,72,561,91.22,Exeggutor
281,281,556,90.41,Bellossom


Common features of the top 20 results include the following:

|Feature|Explaination|
|:------|:-----------|
|Several EX pokemon|Tend to have higher HP and stronger attacks and abilities|
|Magnezone|A high-damage attack that one-hit KOs most pokemon, as well as its pre-evolution accelerating its energy gain|
|Rampardos|High-damage attack that one-hit KOs most pokemon|
|Aerodactly EX|Prevents the active pokemon from evolving, so it stops most pokemon from reaching their full power|

In [38]:
# get losses as a data frame
def losses_df(df, id):
    return df[((df['player1'] == id) & ~(df['p1win'])) | ((df['player2'] == id) & ~(df['p2win']))]

Below, we can test which battles Rampardos lost, and we will see several of the other top-20 win-rate Pokemon present

In [39]:
losses_df(df, 300)[['result', 'p1name', 'p2name']]

,result,p1name,p2name
23092,1,Exeggutor Ex,Rampardos
23400,1,Exeggutor Ex,Rampardos
23708,1,Whimsicott,Rampardos
53584,1,Primape,Rampardos
53892,1,Primape,Rampardos
54200,1,Primape,Rampardos
54508,1,Aerodactyl Ex,Rampardos
60976,1,Lumineon,Rampardos
61284,1,Lumineon,Rampardos
63132,1,Electrode,Rampardos


We can also analyze the top 20 Pokemon with turn-1 win rates. Turn-2 pokemon tend to win more of the battles on average due to beign the first player to start gaining energy, however, top-performing Pokemon with turn-1 win counts will show us which Pokemon value later evolution stages more than energy acceleration.

In [40]:
turn1_top_20 = wins[['id', 'player_win', 'name']].sort_values('player_win', axis=0, ascending=False).head(20)
turn1_top_20

,id,player_win,name
300,300,308,Rampardos
74,74,306,Exeggutor Ex
75,75,303,Exeggutor Ex
176,176,303,Aerodactyl Ex
289,289,301,Magnezone
290,290,300,Magnezone
286,286,287,Infernape Ex
99,99,285,Starmie Ex
72,72,284,Exeggutor
261,261,283,Kabutops


Unsurprisingly, most of these pokemon are also in the top-20 win-rate overall, however there are a few differences in pokemon that can be interesting to look at:

In [41]:
top_difference = [int(id) for id in set(turn1_top_20['id'].unique()) - set(top_20['id'].unique())]

if len(top_difference) == 0:
    print('No Difference in the Top Sets')
else:
    print(turn1_top_20[turn1_top_20['id'].isin(top_difference)][['id', 'name']])

      id        name
150  150  Marowak Ex


### Ties

Below we can analyze all pokemon that ever tied.

In [42]:
df[df['tie']].groupby('p1name')['result'].count().reset_index().sort_values('result', ascending=False)

,p1name,result
7,Lumineon,18
5,Hitmonlee,12
1,Chatot,10
8,Manaphy,10
9,Mespirit,10
3,Ditto,8
10,Mew Ex,8
4,Fan Rotom,7
0,Aerodactyl,4
2,Cinccino,3


Most of these Pokemon have attacks that can only target benched Pokemon, so it is natural that when two bench-attacking-only Pokemon fight each other nobody wins. Additionally, Fan Rotom and Aerodactly have an ability that shuffles the opponent's pokemon back into their hand instead of KOing, so it is also natural that they would likely tie against each other, as well as against a few of the bench-attacking-only Pokemon.

### Near-No Wins

The final anaylsis we will perform is the Pokemon with the lowest win-rate

In [43]:
wins[wins['wins'] <= 5][['id', 'name', 'wins']].sort_values('wins', ascending=False)

,id,name,wins
165,165,Lumineon,1
19,19,Hitmonlee,0
39,39,Chatot,0
48,48,Manaphy,0
52,52,Mespirit,0
166,166,Lumineon,0


These results make sense since a few of them are bench-attacking only, and the others have non-damage-dealing attacks

# Charts

In [44]:
(
    ggplot(
        df,
        aes(
            x='result'
        )
    )
    + geom_bar()
    + scale_x_discrete(breaks=[2, 1, 0], labels=['Player 2', 'Player 1', 'Tie'])
    + labs(
        title='Total Battle Results',
        x='Winner',
        y='Count'
    )
)

In [45]:
(
    ggplot(
        (wins.groupby('is_ex')['wins'].sum() / wins.groupby('is_ex')['wins'].count()).round().reset_index(),
        aes(
            x='is_ex',
            y='wins'
        )
    )
    + geom_bar(stat='identity')
    + scale_x_discrete(breaks=[False, True], labels=['Non-EX', 'EX'])
    + labs(
        title='Average Wins by EX',
        x='EX?',
        y='Avg. Wins',
    )
)

In [46]:
(
    ggplot(
        (wins.groupby('type')['wins'].sum() / wins.groupby('type')['wins'].count()).round().reset_index().sort_values('wins', ascending=True),
        aes(
            x='type',
            y='wins'
        )
    )
    + geom_bar(stat='identity')
    + labs(
        title='Wins by Type',
        x='Type',
        y='Avg. Wins'
    )
)

In [47]:
(
    ggplot(
        (wins.groupby('stage')['wins'].sum() / wins.groupby('stage')['wins'].count()).round().reset_index().sort_values('wins'),
        aes(
            x='stage',
            y='wins'
        )
    )
    + geom_bar(stat='identity')
    + labs(
        title='Wins by Final Evolution Stage',
        x='Stage',
        y='Avg. Wins'
    )
)

In [48]:
(
    ggplot(
        (wins.groupby('has_ability')['wins'].sum() / wins.groupby('has_ability')['wins'].count()).round().reset_index().sort_values('wins'),
        aes(
            x='has_ability',
            y='wins'
        )
    )
    + geom_bar(stat='identity')
    + scale_x_discrete(breaks=[False, True], labels=['No', 'Yes'])
    + labs(
        title='Wins by Having an Ability',
        x='Has Ability?',
        y='Avg. Wins'
    )
)

In [49]:
(
    ggplot(
        wins,
        aes(
            x='hp',
            y='wins'
        )
    )
    + geom_boxplot()
    + geom_point(
        data=wins[wins['ties'] > 0],
        color='red'
    )
    + labs(
        title='Wins by Final Evolution HP',
        x='HP',
        y='Wins'
    )
)

In [50]:
wins[(wins['ties'] > 0) & (wins['hp'] == 80) & (wins['wins'] > 202)]

,id,player_win,player_tie,opponent_win,opponent_tie,wins,ties,losses,name,win_perc,is_ex,pk_index,type,stage,ability,has_ability,hp
66,66,87,7,122,6,209,13,393,Fan Rotom,33.98,False,485,Normal,Basic,None,False,80


In [51]:
wins[(wins['ties'] > 0) & (wins['hp'] == 130) & (wins['wins'] > 357)]

,id,player_win,player_tie,opponent_win,opponent_tie,wins,ties,losses,name,win_perc,is_ex,pk_index,type,stage,ability,has_ability,hp
33,33,189,8,256,7,445,15,155,Mew Ex,72.36,True,298,Psychic,Basic,None,False,130


In [52]:
wins[(wins['ties'] > 0) & (wins['hp'] == 140) & (wins['wins'] > 427)]

,id,player_win,player_tie,opponent_win,opponent_tie,wins,ties,losses,name,win_perc,is_ex,pk_index,type,stage,ability,has_ability,hp
261,261,283,2,255,2,538,4,73,Kabutops,87.48,False,158,Fighting,Stage 2,None,False,140


In [53]:
(
    ggplot(
        (wins.groupby('ability')['wins'].sum() / wins.groupby('ability')['wins'].count()).round().reset_index().sort_values('wins'),
        aes(
            x='ability',
            y='wins'
        )
    )
    + geom_bar(stat='identity')
    + labs(
        title='Wins by Ability',
        x='Ability',
        y='Avg. Wins'
    )
)

# Final Interesting Analysis

Almost half of all pokemon, when facing off against themselves, win when they are the first player.

In [54]:
df[(df['player1'] == df['player2']) & (df['result'] == 1)]

,player1,player2,result,p1name,p2name,p1win,p2win,tie
1854,6,6,1,Articuno,Articuno,True,False,False
5253,17,17,1,Mewtwo Ex,Mewtwo Ex,True,False,False
9888,32,32,1,Dedenne,Dedenne,True,False,False
14523,47,47,1,Palkia Ex,Palkia Ex,True,False,False
19776,64,64,1,Heatran,Heatran,True,False,False
...,...,...,...,...,...,...,...,...
92700,300,300,1,Rampardos,Rampardos,True,False,False
93009,301,301,1,Gallade Ex,Gallade Ex,True,False,False
93627,303,303,1,Bastidon,Bastidon,True,False,False
93936,304,304,1,Garchomp,Garchomp,True,False,False


The only Pokemon to tie against themselves are bench-attacking-only, non-damage-dealing, and Kabutops.

Kaputops makes sense once you realize that its attack deals 50 damage, then heals 50 damage, so they get stuck in a loop of dealing damage to eachother, then healing back to full health.

In [55]:
df[(df['player1'] == df['player2']) & (df['result'] == 0)]

,player1,player2,result,p1name,p2name,p1win,p2win,tie
5871,19,19,0,Hitmonlee,Hitmonlee,False,False,True
8034,26,26,0,Ditto,Ditto,False,False,True
12051,39,39,0,Chatot,Chatot,False,False,True
14832,48,48,0,Manaphy,Manaphy,False,False,True
16068,52,52,0,Mespirit,Mespirit,False,False,True
20394,66,66,0,Fan Rotom,Fan Rotom,False,False,True
50985,165,165,0,Lumineon,Lumineon,False,False,True
51294,166,166,0,Lumineon,Lumineon,False,False,True
80649,261,261,0,Kabutops,Kabutops,False,False,True
